<a href="https://colab.research.google.com/github/atharvasnighot/Tensor/blob/main/Bhojan1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

restaurants = pd.read_csv('/content/sample_data/Swiggy_Nagpur.csv')

restaurants = restaurants[['id', 'Restaurant Name', 'Category', 'Rating', 'Area', 'Locality', 'Veg', 'Long Distance Delivery']]
restaurants.dropna(inplace=True)

# Convert non-string columns to strings
restaurants['Category'] = restaurants['Category'].astype(str)
restaurants['Rating'] = restaurants['Rating'].astype(str)
restaurants['Area'] = restaurants['Area'].str.strip().str.replace(" ", "")
restaurants['Locality'] = restaurants['Locality'].str.replace(" ", "")
restaurants['Veg'] = restaurants['Veg'].astype(str)
restaurants['Long Distance Delivery'] = restaurants['Long Distance Delivery'].astype(str)

# Concatenate the columns into a single string
restaurants['tags'] = restaurants['Category'] + restaurants['Rating'] + restaurants['Area'] + restaurants['Locality'] + restaurants['Veg'] + restaurants['Long Distance Delivery']

# Convert the 'tags' column to lowercase
restaurants['tags'] = restaurants['tags'].str.lower()

# Now, you can use TF-IDF vectorization
tfidf = TfidfVectorizer(max_features=10000, stop_words='english')
vector = tfidf.fit_transform(restaurants['tags']).toarray()

# Create a new DataFrame with 'id', 'Restaurant Name', and vectorized data
new_df = restaurants[['id', 'Restaurant Name']]
new_df = pd.concat([new_df, pd.DataFrame(vector)], axis=1)


In [ ]:
new_df.head()

,id,Restaurant Name,0,1,2,3,4,5,6,7,...,543,544,545,546,547,548,549,550,551,552
0,0.0,Biryani By Kilo,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,Haldirams Sweets & Namkeen,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2.0,Pind Punjabi,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3.0,Behrouz Biryani,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4.0,Jay Ambika Bhojnalay,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
vector[0]

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.49844562, 0.        , 0.        , 0.        , 0.     

In [ ]:
vector


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
!pip install nltk

In [ ]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [ ]:
def stem(text):
  y = []

  for i in text.split():
    y.append(ps.stem(i))

  return " ".join(y)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(vector)
similarity

array([[1.        , 0.09053867, 0.05286053, ..., 0.        , 0.11435022,
        0.07896076],
       [0.09053867, 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.05286053, 0.        , 1.        , ..., 0.        , 0.07622902,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.11435022, 0.        , 0.07622902, ..., 0.        , 1.        ,
        0.        ],
       [0.07896076, 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [ ]:
def recommend(restaurant):
    index = new_df[new_df['Restaurant Name'] == restaurant].index[0]
    distances = sorted(list(enumerate(similarity[index])), reverse=True, key=lambda x: x[1])
    for i in distances[1:6]:
        print(new_df.iloc[i[0]]['Restaurant Name'])


In [ ]:
recommend('Checkers')

Nanking Chinese Restaurant
FSB
Meals101
SAI CHINESE CENTER
Maharaj Bag Lawn


In [24]:
import pickle

In [26]:
pickle.dump(new_df,open('rest_list.pkl','wb'))
pickle.dump(similarity,open('similarity.pkl','wb'))

In [30]:
!pip install streamlit

In [31]:

import pickle
import streamlit as st
import requests

def fetch_poster(movie_id):
    url = "https://api.themoviedb.org/3/movie/{}?api_key=8265bd1679663a7ea12ac168da84d2e8&language=en-US".format(movie_id)
    data = requests.get(url)
    data = data.json()
    poster_path = data['poster_path']
    full_path = "https://image.tmdb.org/t/p/w500/" + poster_path
    return full_path

def recommend(restro):
    index = restaurants[restaurants['Restaurant Name'] == restro].index[0]
    distances = sorted(list(enumerate(similarity[index])), reverse=True, key=lambda x: x[1])
    recommended_restro_names = []
    recommended_restro_posters = []
    for i in distances[1:6]:
        # fetch the movie poster
        movie_id = restaurants.iloc[i[0]].id
        recommended_restro_posters.append(fetch_poster(id))
        recommended_restro_names.append(restaurants.iloc[i[0]].title)

    return recommended_restro_names,recommended_restro_posters


st.header('Restro Recommender System')
movies = pickle.load(open('/content/rest_list.pkl','rb'))
similarity = pickle.load(open('/content/similarity.pkl','rb'))

restro_list = restaurants['Restaurant Name'].values
selected_restro = st.selectbox(
    "Type or select a movie from the dropdown",
    restro_list
)

if st.button('Show Recommendation'):
    recommended_restro_names,recommended_restro_posters = recommend(selected_restro)
    col1, col2, col3, col4, col5 = st.beta_columns(5)
    with col1:
        st.text(recommended_restro_names[0])
        st.image(recommended_restro_posters[0])
    with col2:
        st.text(recommended_restro_names[1])
        st.image(recommended_restro_posters[1])

    with col3:
        st.text(recommended_restro_names[2])
        st.image(recommended_restro_posters[2])
    with col4:
        st.text(recommended_restro_names[3])
        st.image(recommended_restro_posters[3])
    with col5:
        st.text(recommended_restro_names[4])
        st.image(recommended_restro_posters[4])

